### Install required libraries

Run the following cells to install required libraries. Please ignore any compatibility related errors. Please run the cells below only once.

After your install these libraries it is recommended that you **restart** the notebook kernel from the **Kernel** menu above. After restarting the kernel, start from the `Train a deep learning model` section.

In [ ]:
!pip install h5py==2.10.0
!pip install tensorflow==2.0.0
!pip install keras==2.3.1

In [ ]:
!pip install onnxmltools==1.6.1
!pip install keras2onnx==1.6.1
!pip install onnxruntime==1.3.0
!pip install tf2onnx==1.6.3

# Train a deep learning model
In this notebook you will train a deep learning model to classify the descriptions of car components as compliant or non-compliant. 

Each document in the supplied training data set is a short text description of the component as documented by an authorized technician. 
The contents include:
- Manufacture year of the component (e.g. 1985, 2010)
- Condition of the component (poor, fair, good, new)
- Materials used in the component (plastic, carbon fiber, steel, iron)

The compliance regulations dictate:
*Any component manufactured before 1995 or in fair or poor condition or made with plastic or iron is out of compliance.*

For example:
* Manufactured in 1985 made of steel in fair condition -> **Non-compliant**
* Good condition carbon fiber component manufactured in 2010 -> **Compliant**
* Steel component manufactured in 1995 in fair condition -> **Non-Compliant**

The labels present in this data are 0 for compliant, 1 for non-compliant.

The challenge with classifying text data is that deep learning models only undertand vectors (e.g., arrays of numbers) and not text. To encode the car component descriptions as vectors, we use an algorithm from Stanford called [GloVe (Global Vectors for Word Representation)](https://nlp.stanford.edu/projects/glove/). GloVe provides us pre-trained vectors that we can use to convert a string of text into a vector.

### Setup
To begin, you will need to provide the following information about your Azure Subscription.

**If you are using your own Azure subscription, please provide names for subscription_id, resource_group, workspace_name and workspace_region to use.** Note that the workspace needs to be of type [Machine Learning Workspace](https://docs.microsoft.com/en-us/azure/machine-learning/service/setup-create-workspace).

**If an environment is provided to you be sure to replace XXXXX in the values below with your unique identifier.**

In the following cell, be sure to set the values for `subscription_id`, `resource_group`, `workspace_name` and `workspace_region` as directed by the comments (*these values can be acquired from the Azure Portal*).

To get these values, do the following:
1. Navigate to the Azure Portal and login with the credentials provided.
2. From the left hand menu, under Favorites, select `Resource Groups`.
3. In the list, select the resource group with the name similar to `XXXXX`.
4. From the Overview tab, capture the desired values.

Execute the following cell by selecting the `>|Run` button in the command bar above.

In [ ]:
#Provide the Subscription ID of your existing Azure subscription
subscription_id = "" # <- needs to be the subscription with the MCW-MLOps resource group

#Provide values for the existing Resource Group 
resource_group = "MCW-MLOps-XXXXX" # <- replace XXXXX with your unique identifier

#Provide the Workspace Name and Azure Region of the Azure Machine Learning Workspace
workspace_name = "quick-starts-ws-XXXXX" # <- replace XXXXX with your unique identifier
workspace_region = "eastus" # <- region of your Quick-Starts resource group

In [ ]:
experiment_name = 'deep-learning'
project_folder = './dl'
deployment_folder = './deploy'
datasets_folder = './datasets'
onnx_export_folder = 'onnx'

# this is the URL to the CSV file containing the car component descriptions
cardata_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
            'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/'
            'quickstarts/connected-car-data/connected-car_components.csv')



cardata_ds_name = 'connected_car_components'
cardata_ds_description = 'Connected car components data'

# this is the name of the AML Compute cluster
cluster_name = "amlcluster"

embedding_dim = 100                                        
training_samples = 90000                                 
validation_samples = 5000    
max_words = 10000

# Create the Azure Machine Learning resources

The Azure Machine Learning SDK provides a comprehensive set of a capabilities that you can use directly within a notebook including:
- Creating a **Workspace** that acts as the root object to organize all artifacts and resources used by Azure Machine Learning.
- Creating **Experiments** in your Workspace that capture versions of the trained model along with any desired model performance telemetry. Each time you train a model and evaluate its results, you can capture that run (model and telemetry) within an Experiment.
- Creating **Compute** resources that can be used to scale out model training, so that while your notebook may be running in a lightweight container in a notebook environment, your model training can actually occur on a powerful cluster that can provide large amounts of memory, CPU or GPU. 
- Using **Automated Machine Learning (AutoML)** to automatically train multiple versions of a model using a mix of different ways to prepare the data and different algorithms and hyperparameters (algorithm settings) in search of the model that performs best according to a performance metric that you specify.
- Packaging a Docker **Image** that contains everything your trained model needs for scoring (prediction) in order to run as a web service.
- Deploying your Image to either Azure Kubernetes or Azure Container Instances, effectively hosting the **Web Service**.

In the Azure Machine Learning compute instance you created, all of the libraries needed for Azure Machine Learning are pre-installed. To use them, you just need to import them. Run the following cell to do so:

In [ ]:
import logging
import os
import json

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.model import Model
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.dnn import TensorFlow

import keras
import tensorflow
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import onnxruntime

print("keras version: {} tensorflow version: {}".format(keras.__version__, tensorflow.__version__))
print("Azure ML SDK version:", azureml.core.VERSION)

## Create and connect to an Azure Machine Learning Workspace
Run the following cell to create a new Azure Machine Learning **Workspace** and save the configuration to disk (next to the Jupyter notebook). 

**Important Note**: You will be prompted to login in the text that is output below the cell. Be sure to navigate to the URL displayed and enter the code that is provided. Once you have entered the code, return to this notebook and wait for the output to read `Workspace configuration succeeded`.

In [ ]:
# By using the exist_ok param, if the worskpace already exists you get a reference to the existing workspace
# allowing you to re-run this cell multiple times as desired (which is fairly common in notebooks).
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

ws.write_config()
print('Workspace configuration succeeded')

### Create AML Compute Cluster
Now you are ready to create the CPU compute cluster. Run the following cell to create a new compute cluster (or retrieve the existing cluster if it already exists). The code below will create a *CPU based* cluster where each node in the cluster is of the size `Standard_D2_V2`, and the cluster is restricted to use 1 node. This will take couple of minutes to create.

In [ ]:
### Create AML CPU based Compute Cluster

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           min_nodes=1, max_nodes=1)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current AmlCompute. 
print(compute_target.status.serialize())

### Create and register input Datasets for training

In [ ]:
# this is the URL to the CSV file containing the GloVe vectors
glove_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
             'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/'
             'quickstarts/connected-car-data/glove.6B.100d.txt')

glove_ds_name = 'glove_6B_100d'
glove_ds_description ='GloVe embeddings 6B 100d'

# this is the URL to the CSV file containing the care component descriptions
cardata_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
            'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/'
            'quickstarts/connected-car-data/connected-car_components.csv')

cardata_ds_name = 'connected_car_components'
cardata_ds_description = 'Connected car components data'

try:
    glove_ds = Dataset.get_by_name(workspace=ws, name=glove_ds_name)
    print('GloVe embeddings dataset already registered.')
except:
    print('Registering GloVe embeddings dataset...')
    glove_ds = Dataset.File.from_files(glove_url)
    glove_ds.register(workspace=ws, name=glove_ds_name, description=glove_ds_description)
    print('GloVe embeddings dataset successfully registered.')

try:
    cardata_ds = Dataset.get_by_name(workspace=ws, name=cardata_ds_name)
    print('Connected car components dataset already registered.')
except:
    print('Registering connected car components dataset...')
    cardata_ds = Dataset.Tabular.from_delimited_files(path=cardata_url)
    cardata_ds.register(workspace=ws, name=cardata_ds_name, description=cardata_ds_description)
    print('Connected car components dataset successfully registered.')

### Create the ScriptRunConfig with custom TensforFlow 2.0 Enviroment

In [ ]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies
tensorflow_env = Environment.get(workspace=ws, name='AzureML-TensorFlow-2.0-CPU').clone('Custom-TensforFlow-Env')
cd = tensorflow_env.python.conda_dependencies
cd.add_pip_package('h5py==2.10.0')
cd.add_pip_package('pandas==1.1.2')
cd.add_pip_package('keras==2.3.1')
cd.add_pip_package('onnxmltools==1.6.1')
cd.add_pip_package('keras2onnx==1.6.1')
cd.add_pip_package('onnxruntime==1.3.0')
cd.add_pip_package('tf2onnx==1.6.3')
tensorflow_env.register(workspace=ws)

In [ ]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=project_folder, 
                      script='train.py', 
                      arguments=['--glove_ds', glove_ds.as_named_input('glove').as_mount(), 
                                 '--cardata_ds_name', cardata_ds_name], 
                      compute_target=compute_target, 
                      environment=tensorflow_env)

## Remotely train a deep learning model using the Azure ML Compute
In the following cells, you will *not* train the model against the data you just downloaded using the resources provided by your notebook selected compute instances. Instead, you will deploy an Azure ML Compute cluster that will download the data and use a trainings script to train the model. In other words, all of the training will be performed remotely with respect to this notebook. 



In [ ]:
# create project folder
if not os.path.exists(project_folder):
    os.makedirs(project_folder)

### Create the training script

In [ ]:
%%writefile $project_folder/train.py

import os
import argparse
import numpy as np
import pandas as pd

import tensorflow
import keras
from keras import models 
from keras import layers
from keras import optimizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

from azureml.core import Run
from azureml.core.dataset import Dataset

import sys
print('Python version:', sys.version)
print("keras version: {} tensorflow version: {}".format(keras.__version__, tensorflow.__version__))

parser = argparse.ArgumentParser("train")
parser.add_argument("--glove_ds", type=str, help="glove dataset input mount path", dest="glove_ds", required=True)
parser.add_argument("--cardata_ds_name", type=str, help="dataset name", dest="cardata_ds_name", required=True)

args = parser.parse_args()
print("Argument 1: %s" % args.glove_ds)
print("Argument 2: %s" % args.cardata_ds_name)

glove_file_path = args.glove_ds
cardata_ds_name = args.cardata_ds_name

embedding_dim = 100                                        
training_samples = 90000                                 
validation_samples = 5000    
max_words = 10000

run = Run.get_context()
ws = run.experiment.workspace

print("Loading car components data...")
cardata_ds = Dataset.get_by_name(workspace=ws, name=cardata_ds_name)
car_components_df = cardata_ds.to_pandas_dataframe()
components = car_components_df["text"].tolist()
labels = car_components_df["label"].tolist()
print("Loading car components data completed.")

# use the Tokenizer from Keras to "learn" a vocabulary from the entire car components text
print("Tokenizing data...")    

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(components)
sequences = tokenizer.texts_to_sequences(components)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=embedding_dim)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])  
np.random.seed(100)
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]

x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

x_test = data[training_samples + validation_samples:]
y_test = labels[training_samples + validation_samples:]
print("Tokenizing data complete.")

# apply the vectors provided by GloVe to create a word embedding matrix
print("Applying GloVe vectors...")

embeddings_index = {}
f = open(glove_file_path)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector    
print("Applying GloVe vectors compelted.")

# use Keras to define the structure of the deep neural network   
print("Creating model structure...")

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=embedding_dim))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# fix the weights for the first layer to those provided by the embedding matrix
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
print("Creating model structure completed.")

opt = optimizers.RMSprop(lr=0.1)

print("Training model...")
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=3, 
                    batch_size=32,
                    validation_data=(x_val, y_val))
print("Training model completed.")


print("Saving model files...")
# create a ./outputs/model folder in the compute target
# files saved in the "./outputs" folder are automatically uploaded into run history
path = os.path.join('./outputs', 'model')
os.makedirs(path, exist_ok=True)
# save model h5
model.save(os.path.join(path, 'model.h5'))

print("model saved in ./outputs/model folder")
print("Saving model files completed.")

import onnxmltools
import os

# Convert the Keras model to ONNX
onnx_model_name = 'model.onnx'
converted_model = onnxmltools.convert_keras(model, onnx_model_name, target_opset=7)

# Save the model locally...
onnxmltools.utils.save_model(converted_model, os.path.join(path, 'model.onnx'))
print('Model exported in ONNX format...')


## Submit the training run

The code pattern to submit a training run to Azure Machine Learning compute targets is always:

- Create an experiment to run.
- Submit the experiment.
- Wait for the run to complete.

### Create the experiment

In [ ]:
experiment = Experiment(ws, experiment_name)

### Submit the experiment

In [ ]:
run = experiment.submit(config=src)

Wait for the run to complete by executing the following cell. Note that this process will perform the following:
- Build and deploy the container to Azure Machine Learning compute (~8 minutes)
- Execute the training script (~2 minutes)

If you change only the training script and re-submit, it will run faster the second time because the necessary container is already prepared so the time requried is just that for executing the training script.

*Run the cell below and wait till the **Run Status** is **Completed** before proceeding ahead*

In [ ]:
RunDetails(run).show()

## Download the model files from the run

In the training script, the Keras model is saved into two files, model.json and model.h5, in the outputs/models folder on the CPU cluster AmlCompute node. Azure ML automatically uploaded anything written in the ./outputs folder into run history file store. Subsequently, we can use the run object to download the model files. They are under the the outputs/model folder in the run history file store, and are downloaded into a local folder named model.

In [ ]:
# create a model folder in the current directory
os.makedirs('model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)
        print('Download completed.')

## Restore the model from model.h5 file

In [ ]:
model = load_model('./model/model.h5')
print("Model loaded from disk.")
print(model.summary())

## Evaluate the model on test data

You can also evaluate how accurately the model performs against data it has not seen. Run the following cell to load the test data that was not used in either training or evaluating the model. 
Run the following cell to see the accuracy on the test set (it is the second number in the array displayed, on a scale from 0 to 1).

In [ ]:
embedding_dim = 100                                        
training_samples = 90000                                 
validation_samples = 5000    
max_words = 10000

cardata_ds_name = 'connected_car_components'
cardata_ds = Dataset.get_by_name(workspace=ws, name=cardata_ds_name)

car_components_df = cardata_ds.to_pandas_dataframe()
components = car_components_df["text"].tolist()
labels = car_components_df["label"].tolist()

# use the Tokenizer from Keras to "learn" a vocabulary from the entire car components text
print("Tokenizing data...") 

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(components)
sequences = tokenizer.texts_to_sequences(components)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=embedding_dim)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])  
np.random.seed(100)
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]

x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

x_test = data[training_samples + validation_samples:]
y_test = labels[training_samples + validation_samples:]
print("Tokenizing data complete.")

In [ ]:
print('Model evaluation will print the following metrics: ', model.metrics_names)
evaluation_metrics = model.evaluate(x_test, y_test)
print(evaluation_metrics)

Log the evaluation metrics to the experiment **Run**

In [ ]:
run.log(model.metrics_names[0], evaluation_metrics[0], 'Model test data loss')
run.log(model.metrics_names[1], evaluation_metrics[1], 'Model test data accuracy')

**Save the run information to json file**

In [ ]:
run_info = {}
run_info["id"] = run.id

print("Saving run_info.json...")
os.makedirs('./outputs', exist_ok=True)
run_info_filepath = os.path.join('./outputs', 'run_info.json')
with open(run_info_filepath, "w") as f:
    json.dump(run_info, f)

## Converting a Keras model to ONNX

In the steps that follow, you will convert Keras model you just trained to the ONNX format. This will enable you to use this model for classification in a very broad range of environments, outside of Azure Databricks including:

Web services
iOS and Android mobile apps
Windows apps
IoT devices
Convert the model to ONNX by running the following cell.

In [ ]:
import onnxmltools
import os

# Convert the Keras model to ONNX
onnx_model_name = 'component_compliance.onnx'
converted_model = onnxmltools.convert_keras(model, onnx_model_name, target_opset=7)

# Save the model locally...
onnx_model_path = os.path.join(deployment_folder, onnx_export_folder)
os.makedirs(onnx_model_path, exist_ok=True)
onnxmltools.utils.save_model(converted_model, os.path.join(onnx_model_path,onnx_model_name))
print('Model exported in ONNX format...')

The above cell created a new file called component_compliance.onnx that contains the ONNX version of the model.

Now try using this ONNX model to classify a component description by running the following cell. Remeber the prediction will be a value close to 0 (non-compliant) or to 1 (compliant).

## Compare ONNX Inference Performace with Keras

Create an onnxruntime InferenceSession and observe the expected input shape for inference. Classify a sample data from test set using both ONNX and Keras. Remeber the prediction will be a value close to 0 (non-compliant) or to 1 (compliant).

Next, we will evaluate the performance of ONNX and Keras by running the same sample 10,000 times. You will observe that ONNX is approximately 10 times faster than Keras in making inferences.

In [ ]:
import onnxruntime
# Load the ONNX model and observe the expected input shape
onnx_session = onnxruntime.InferenceSession(
    os.path.join(os.path.join(deployment_folder, onnx_export_folder), onnx_model_name))
input_name = onnx_session.get_inputs()[0].name
output_name = onnx_session.get_outputs()[0].name
print('Expected input shape: ', onnx_session.get_inputs()[0].shape)

In [ ]:
# Load the car components labeled data
print("Loading car components data...")

cardata_ds = Dataset.get_by_name(workspace=ws, name=cardata_ds_name)
car_components_df = cardata_ds.to_pandas_dataframe()
components = car_components_df["text"].tolist()
labels = car_components_df["label"].tolist()

print("Loading car components data completed.")   

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(components)
sequences = tokenizer.texts_to_sequences(components)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=embedding_dim)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])      
np.random.seed(100)
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_test = data[training_samples + validation_samples:]
y_test = labels[training_samples + validation_samples:]

In [ ]:
# Grab one sample from the test data set
x_test_float = np.reshape(x_test[1502].astype(np.float32), (1,100))

print('Input shape: ', x_test_float.shape)
print('ONNX prediction: ', onnx_session.run([output_name], {input_name: x_test_float}))

# Use Keras to make predictions on the same sample
print('Keras prediction: ', model.predict(x_test_float))

# Next we will compare the performance of ONNX vs Keras
import timeit
n = 100000

In [ ]:
start_time = timeit.default_timer()
for i in range(n):
    model.predict(x_test_float)
keras_elapsed = timeit.default_timer() - start_time
print('Keras performance: ', keras_elapsed)

In [ ]:
start_time = timeit.default_timer()
for i in range(n):
    onnx_session.run([output_name], {input_name : x_test_float})
onnx_elapsed = timeit.default_timer() - start_time
print('ONNX performance: ', onnx_elapsed)
print('ONNX is about {} times faster than Keras'.format(round(keras_elapsed/onnx_elapsed)))